# What is an API? 🤔

Glad you asked.

We now have the ability to exchange data between applications using HyperText Transport Protocol (HTTP) and a way to represent complex data that we are sending back and forth between these applications using eXtensible Markup Language (XML) or JavaScript Object Notation (JSON).

The next step is to begin to define and document "contracts" between applications using these techniques. The general name for these application-to-application contracts is Application Program Interfaces or APIs. When we use an API, generally one program makes a set of services available for use by other applications and publishes the APIs (i.e., the "rules") that must be followed to access the services provided by the program.

When we begin to build our programs where the functionality of our program includes access to services provided by other programs, we call the approach a Service-Oriented Architecture or SOA. A SOA approach is one where our overall application makes use of the services of other applications. A non-SOA approach is where the application is a single standalone application which contains all of the code necessary to implement the application.

We see many examples of SOA when we use the web. We can go to a single web site and book air travel, hotels, and automobiles all from a single site. The data for hotels is not stored on the airline computers. Instead, the airline computers contact the services on the hotel computers and retrieve the hotel data and present it to the user. When the user agrees to make a hotel reservation using the airline site, the airline site uses another web service on the hotel systems to actually make the reservation. And when it comes time to charge your credit card for the whole transaction, still other computers become involved in the process.

![](assets/api.png)

![](assets/api2.png)


You can check this [video](https://youtu.be/s7wmiS2mSXY) if the above description is not very clear. Actually, check it anyway!

### Querying APIs

We will be mianly doing `GET` requests to query data from an API.
For this first example we will use the [OpenNotify API](http://open-notify.org/Open-Notify-API/).

First we need to know the API endpoints. That is a server route that is used to retrieve the data from the API. This points to where the data is located in the API. 

For example the [Reddit API](https://www.reddit.com/dev/api/) has a `/comments` endpoint from whcih you can get information about comments while the `/user` endpoint is used to get information about the users. 
To access this data you would need the base url (e.g http://www.reddit.com/) and the endpoint which is added to the base url (e.g. 'http://www.reddit.com/user/spilcm/comments/.json).

Going back the example. We will look at OpenNotify `iss-now.json` endpoint. Which gives us the current latitute and lingitude of the International Space Station. 


In [3]:
import requests

response = requests.get("http://api.open-notify.org/iss-now.json")

print(response.status_code)

200


What happens if you pass an endpoint that does not exist?

In [4]:
response = requests.get("http://api.open-notify.org/iss-pass")
print(response.status_code)

404


The problem with the above query is that we forgot to add the `.json` but to the end. Let's fix this

In [6]:
response = requests.get("http://api.open-notify.org/iss-pass.json")
print(response.status_code)

400


### Query parameters

We got a `400` response. By looking at the [documentation](http://open-notify.org/Open-Notify-API/ISS-Pass-Times/) we realize we need to pass the coordinates of the location we are interested in by providing the latitude and longitude. 

This is done by passing an optional keyword argument `params` to our request.

We can make a dictionary with the `lat` and `lon` (latitute and longitude, respectively) and pass them to the `request.get` function.

Let's use the coordinates for New York city and see what we get. 

In [26]:
# Set up the parameters we want to pass to the API.
# This is the latitude and longitude of New York City.
parameters = {"lat": 40.71, "lon": -74}

# Make a get request with the parameters.
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

# Print the content of the response (the data the server returned)
print(response.content)

# This gets the same data as the command above
response = requests.get("http://api.open-notify.org/iss-pass.json?lat=40.71&lon=-74")
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1526981162, \n    "latitude": 40.71, \n    "longitude": -74.0, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 554, \n      "risetime": 1527033468\n    }, \n    {\n      "duration": 638, \n      "risetime": 1527039195\n    }, \n    {\n      "duration": 574, \n      "risetime": 1527045047\n    }, \n    {\n      "duration": 548, \n      "risetime": 1527050911\n    }, \n    {\n      "duration": 615, \n      "risetime": 1527056715\n    }\n  ]\n}\n'
b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1526981162, \n    "latitude": 40.71, \n    "longitude": -74.0, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 554, \n      "risetime": 1527033468\n    }, \n    {\n      "duration": 638, \n      "risetime": 1527039195\n    }, \n    {\n      "duration": 574, \n      "risetime": 1527045047\n    }, \n    {\n      "duration": 548, \n      "r

If you rememeber from the previous section we can place this information in a json object and then use this as a Python object. 

In [21]:
response_json = response.json()
print(response_json)
print(f'\n \n The type of the response is {type(response_json)}')

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1526981162, 'latitude': 40.71, 'longitude': -74.0, 'passes': 5}, 'response': [{'duration': 554, 'risetime': 1527033468}, {'duration': 638, 'risetime': 1527039195}, {'duration': 574, 'risetime': 1527045047}, {'duration': 548, 'risetime': 1527050911}, {'duration': 615, 'risetime': 1527056715}]}

 
 The type of the response is <class 'dict'>


### Content type

The server doesn't just send a status code and the data when it generates a response. It also sends metadata containing information on how the data was generated and how to decode it. This is stored in the response headers. In Python, we can access this with the `headers` property of a response object.

The headers will be shown as a dictionary. Within the headers, `content-type` is the most important key for now. It tells us the format of the response, and how to decode it. For the OpenNotify API, the format is JSON, which is why we could decode it with the `json` package earlier.

In [24]:
# Headers is a dictionary
print(f'{response.headers} \n\n\'')

# Get the content-type from the dictionary.
print(response.headers["content-type"])

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 22 May 2018 09:40:39 GMT', 'Content-Type': 'application/json', 'Content-Length': '519', 'Connection': 'keep-alive', 'Via': '1.1 vegur'} 

'
application/json


>#### Exercise
> a) Can you figure out how to get the number of people in the space?
> Make sure to check the documentation from OPenNotify. Print out the number only
> b) From the response above now print the names of the people in space

In [25]:

# Get the response from the API endpoint.
response = requests.get("http://api.open-notify.org/astros.json")
data = response.json()

print(data)

{'people': [{'name': 'Anton Shkaplerov', 'craft': 'ISS'}, {'name': 'Scott Tingle', 'craft': 'ISS'}, {'name': 'Norishige Kanai', 'craft': 'ISS'}, {'name': 'Oleg Artemyev', 'craft': 'ISS'}, {'name': 'Andrew Feustel', 'craft': 'ISS'}, {'name': 'Richard Arnold', 'craft': 'ISS'}], 'number': 6, 'message': 'success'}


## Accessing Twitter API using tweepy

You can find the documentation for the Twitter API [here](https://developer.twitter.com/en/docs)

To access the Twitter data you need the following:
- Twitter account
- Twitter developer account
- Create an application (get access and cosumer tokens)

If you need help with any of these steps follow this [tutorial](http://docs.inboundnow.com/guide/create-twitter-application/)

<br>
<div class='warn'>
 ⚠️ Make sure to never share your credentials in GitHub
</div>

<br>

We will be using a Python library called Tweepy, this will help us to access the API more efficiently while getting Python ready objects.


As opposed to the previous examples, we first need to authenticate to the API using OAuth. (Check the Tweepy documentation[here](http://tweepy.readthedocs.io/en/v3.6.0/)

In [29]:
import tweepy

consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_token_secret = 'xxx'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
twitter_api = tweepy.API(auth)

print('Logged in as {}'.format(twitter_api.me().name))

Logged in as ✨ 𝕋𝕒𝕟𝕚𝕒  💀👩🏻‍💻🇬🇧🇲🇽


## Querying the API

Now that we are logged in, it means we can now start querying the API. Let's start by collecting out timeline. 

Again, we will be using the Tweepy library, but under the hood this is a `requests.get` function. For this we are going to create a function `see_timeline`. So we can call this multiple times if needed.


Also... tweets contain a lot of information. Let's see what is in a `tweet json object` https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

For this example we will extract the `user.name` Which is the Tweeter's name and the `tweet.text` which is the content of the tweet itself.

In [34]:
def see_timeline(no_tweets):
    """Print no_tweets tweets in our timeline
    Args:
    ----
    no_tweets: int number of tweets to be printed
    """
    for tweet in tweepy.Cursor(twitter_api.home_timeline).items(no_tweets):
        print("\n {} tweeted by {}".format(tweet.text, tweet.user.name))

In [35]:
see_timeline(10)


 New preprint on "Heterogeneous Multi-output Gaussian Process Prediction". Work with @pablorenoz (visiting student a… https://t.co/jyRNlbWrfV tweeted by Mauricio A. Álvarez

 RT @nick_fildes: #Unbreakable #Manchester 🐝🐝🐝 @CrazyPedros 💪💪 https://t.co/m5jXiGC4f2 tweeted by Crazy Pedro

 RT @SlashDataHQ: #DeveloperMarketing – what is it, why is it important, how to develop it successfully?  @jeffsand shared his experiences w… tweeted by Jeff Sandquist

 Benefit sanctions found to be ineffective and damaging

https://t.co/Pky2EfU8S8 tweeted by Juanjo Medina

 Chromocenters, comprising satellite DNA and proteins, help contain DNA inside the nucleus between cell divisions… https://t.co/8JRUWysLNf tweeted by eLife - the journal

 RT @MartinSFP: Interesting point: Facebook has a surprisingly stable c-suite https://t.co/eSQjYukoHS tweeted by 𝖯𝖾𝗇𝗇𝗒 𝖢𝖲 𝖠𝗇𝖽𝗋𝖾𝗐𝗌 🌈🔥

 Uniqlo are asking if I’m resort-ready. Buy me a holiday, corporations, don’t sell me shorts. tweeted by 𝖯𝖾𝗇𝗇𝗒 𝖢𝖲 𝖠𝗇𝖽𝗋𝖾𝗐𝗌 🌈🔥

 RT @Ds

Tweepy returns a cursor object, which makes it easy to collect and store the tweets. 

Now How would you go  about colleting, let's say tweets containing a certain keyword or hashtag? Let's find out

In [48]:
def collect_tweets(keyword):
    """This function will take the query terms for the form submitted by the user. The
    queries can be simple words, hashtags and or a mixture of both.
    It can receive multiple keywords at a time.
    Once the query is submitted via the form, the function access the API and retrieves
    the tweets which are stored in a list"""
    keyword = keyword.strip()
    print(f'Finding tweets containing:: {keyword}')
    tweets = twitter_api.search(keyword)
    
    return tweets

In [49]:
tweets = collect_tweets('hello world')

Finding tweets containing:: hello world


In [50]:
tweets.count

15

In [51]:
for tweet in tweets:
    print (f'{tweet.text} \n')

RT @chiyomaru5pb: Hello Palmer, 
I was thrilled to find out that you use Okabe from S;G which I created, as your profile picture. I own thr… 

RT @Petit__Rien: Hello world have a great day_______🌸 https://t.co/XI67XTp3g7 

RT @Petit__Rien: Hello world have a great day_______🌸 https://t.co/XI67XTp3g7 

Oh hello world! https://t.co/FR8OXwsEge 

@PanasonicUK hello there. I have the 2512 breadmaker, can you recommend which of the dough settings would be the be… https://t.co/Ppub5xtpMj 

RT @hoxeokie: hello i'd like to direct your attention to 5 seconds of pure bliss. jst look at him. look at this angel. give him The Whole W… 

hello world na mayward Family watch niyo po i share ko lang po ito sa inyo ..nawa may tanunuhan kayo sa pg share ko… https://t.co/yhJgQgV3C1 

HELLO GUYS

just a reminder to all of you that even though we did not achieve the world record on the greatest numb… https://t.co/yBajKbnCVI 

RT @matthewwarren: I'm excited to be giving this talk at #prognet London in Septemb

In [22]:
from IPython.core.display import HTML


def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()